In [1]:
!pwd

/Users/apple/Desktop/Insight/Project/repo/taxi_LocID_convert


In [2]:
# pyshp 2.1.0 project description: https://pypi.org/project/pyshp/
!pip install pyshp
# pyproj 2.4.2
!pip install pyproj

In [3]:
import pandas as pd
import numpy as np
import shapefile

In [4]:
sf = shapefile.Reader("taxi_zones/taxi_zones.shp")
fields_name = [field[0] for field in sf.fields[1:]]
shp_dic = dict(zip(fields_name, list(range(len(fields_name)))))
attributes = sf.records()
shp_attr = [dict(zip(fields_name, attr)) for attr in attributes]

In [5]:
def get_lat_lon(sf):
    content = []
    for sr in sf.shapeRecords():
        shape = sr.shape
        rec = sr.record
        loc_id = rec[shp_dic['location_i']]
        
        x = (shape.bbox[0]+shape.bbox[2])/2
        y = (shape.bbox[1]+shape.bbox[3])/2
        
        content.append((loc_id, x, y))
    return pd.DataFrame(content, columns=["location_i", "longitude", "latitude"])

In [6]:
df_loc = pd.DataFrame(shp_attr).join(get_lat_lon(sf).set_index("location_i"), on="location_i")
df_loc = df_loc[['location_i','longitude','latitude','borough','zone','shape_area','shape_leng']]
df_loc.head()

,location_i,longitude,latitude,borough,zone,shape_area,shape_leng
0,1.0,-74.171533,40.689483,EWR,Newark Airport,0.000782,0.116357
1,2.0,-73.822478,40.610824,Queens,Jamaica Bay,0.004866,0.433470
2,3.0,-73.844953,40.865747,Bronx,Allerton/Pelham Gardens,0.000314,0.084341
3,4.0,-73.977725,40.724137,Manhattan,Alphabet City,0.000112,0.043567
4,5.0,-74.187558,40.550664,Staten Island,Arden Heights,0.000498,0.092146


In [7]:
df_loc.to_csv('taxi_locID_lon_lat.csv', index = False, header = True)